<a href="https://colab.research.google.com/github/Alena-Grebeniuk/ML-and-DA/blob/main/Lab07_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testing part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file ’housing_features_description.txt’ for more information on the attributes.

In [1]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### NOTE: The boston dataset has an ethical problem. More on this can e.g. be found in the scikit documentation. ###
### Summary: The dataset contains the proportion of black people, which was assumed that racial self-segregation had a positive impact on house prices. ###


data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
boston_data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
boston_target = raw_df.values[1::2, 2]

df=pd.DataFrame(boston_data)
df.columns=['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X=boston_data
y=boston_target


In [2]:
df.head(10)

,crime_rate,res_land_zoned,industry,charles_river,nox,avg_num_rooms,prop_bf_1940,dst_emply_center,rd_highway_idx,tax_rate,stdnt_tchr_ratio,prop_blacks,low_status_pct
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


# Note 1:

Think about the ethical aspects of this dataset and machine learning in general.

Can you always trust your data source? Can we use every possible information for our models?

### Exercise 1

Write the *split_train_test(X,y,split,seed)*, given an instance matrix $X \in \mathbb{R}^{N \times D}$, labels $y \in Y^N$, a split ratio in $[0, 1]$ and a random seed $\in \mathbb{Z}$. Split the dataset in $(split×100)\%$ of the instances for training our model and the rest for testing, i.e.

$$ \left|X_{\text{train}}\right| = \lceil \text{split} \cdot N \rceil, \qquad |X_{\text{train}}| + |X_{\text{test}}| = N. $$
Make sure you use the given random number generator seed so we all get the same results. The function is supposed to return:

- X_train, y_train: the training instances and labels;
- X_test, y_test: the test instances and labels,

in the same order as was mentioned.

Hint: It may be helpful to use shuffling functionality (e.g. np.random.shuffle).

In [4]:
import numpy as np

def split_train_test(X, y, split, seed):
    np.random.seed(seed)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    split_point = int(np.ceil(split * X.shape[0]))
    train_indices = indices[:split_point]
    test_indices = indices[split_point:]

    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]

    return X_train, y_train, X_test, y_test

### Exercise 2

Write the function *train_linear_reg(X_train,y_train,lmbd)*.
Implement the ridge regression model (slide 24). The function should output the learned weight vector $\theta \in \mathbb{R}^D$ or $\mathbb{R}^{D+1}$ (depending on whether you are adding *bias*).

In [5]:
def train_linear_reg(X_train, y_train, lmbd):
    X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
    D = X_train.shape[1] # Number of features including bias term

    regularization_matrix = lmbd * np.eye(D)
    regularization_matrix[0, 0] = 0 # Do not regularize the bias term

    theta = np.linalg.inv(X_train.T @ X_train + regularization_matrix) @ X_train.T @ y_train

    return theta

### Exercise 3

Write the function *predict(X,theta)* which predicts housing values vector pred for a dataset X and a previously trained parameter vector $\theta$.

In [6]:
def predict(X, theta):
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    y_pred = X @ theta
    return y_pred

### Exercise 4

Write the function *mean_abs_loss(y_true,y_pred)* which computes the mean of the absolute differences between our prediction vector $y\_pred$ and the real housing values $y\_true$.

In [7]:
def mean_abs_loss(y_true, y_pred):
    loss = np.mean(np.abs(y_true - y_pred))
    return loss

### Exercise 5

Evaluate your solutions by running the following code.

Moreover, answer the following questions: What is the most important feature in your model? Are there features that are not so important? What happens if you remove them? Are there outliers with a high absolute loss?

In [8]:
seed = 3
lmbd=1
split=0.7
X_train,y_train,X_test,y_test=split_train_test(X,y,split,seed)
theta=train_linear_reg(X_train,y_train,lmbd)
y_pred=predict(X_test,theta)
mae=mean_abs_loss(y_test,y_pred)
print ('The mean absolute loss is {loss:0.3f}'.format(loss=mae*1000))

The mean absolute loss is 3428.558


Let's evaluate the model using the provided code and analyze the results

In [9]:
# Seed, lambda, and split ratio as given
seed = 3
lmbd = 1
split = 0.7

# Split the dataset
X_train, y_train, X_test, y_test = split_train_test(X, y, split, seed)

# Train the model
theta = train_linear_reg(X_train, y_train, lmbd)

# Make predictions
y_pred = predict(X_test, theta)

# Calculate the mean absolute loss
mae = mean_abs_loss(y_test, y_pred)
print(f'The mean absolute loss is {mae * 1000:.3f}')

The mean absolute loss is 3428.558


Further Analysis
To address the questions in Exercise 5:

Important Features
We can use the magnitude of the weights in theta to determine the importance of each feature. Larger absolute values of weights suggest more influence.

In [10]:
importances = np.abs(theta[1:]) # Ignoring the bias term

# Identifying the most and least important features
most_important_feature_index = np.argmax(importances)
least_important_feature_index = np.argmin(importances)

print(f'The most important feature is at index {most_important_feature_index} with importance {importances[most_important_feature_index]:.3f}')
print(f'The least important feature is at index {least_important_feature_index} with importance {importances[least_important_feature_index]:.3f}')

The most important feature is at index 4 with importance 10.193
The least important feature is at index 6 with importance 0.010


Removing Less Important Features
To evaluate the effect of removing less important features, we can retrain the model without those features and re-evaluate the mean absolute loss.

Outliers
We examine instances where the absolute error is particularly high:

In [11]:
# Calculate absolute errors
absolute_errors = np.abs(y_test - y_pred)

# Identify outliers
outlier_threshold = np.percentile(absolute_errors, 95)
outliers = np.where(absolute_errors > outlier_threshold)

print(f'Indices of outliers: {outliers}')
print(f'Absolute losses of outliers: {absolute_errors[outliers]}')

Indices of outliers: (array([34, 36, 63, 64, 76, 85, 90, 95]),)
Absolute losses of outliers: [26.49525149  8.80863743 10.36662402 11.24269888  9.65223585 12.35100196
 24.10618866 10.99480105]
